In [1]:
import torch
import tqdm

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

if (torch.cuda.is_available()):
    print("Number of devices torch cuda uses: ", torch.cuda.device_count())
    curr_div_ind = torch.cuda.current_device()
    print("Name of the current device used: ", torch.cuda.get_device_name(curr_div_ind))
    print("Properties of the current device: \n", torch.cuda.get_device_properties(curr_div_ind))
    

Using cuda device
Number of devices torch cuda uses:  1
Name of the current device used:  NVIDIA GeForce GTX 1650
Properties of the current device: 
 _CudaDeviceProperties(name='NVIDIA GeForce GTX 1650', major=7, minor=5, total_memory=4095MB, multi_processor_count=14)


In [3]:
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertModel
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score

C:\Users\hp\.conda\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
pt_model = "bert-base-uncased"

In [5]:
tokenizer = BertTokenizer.from_pretrained(pt_model)
bert_model = BertModel.from_pretrained(pt_model)
bert_model.to(device)
bert_model.device

device(type='cuda', index=0)

In [6]:

class Custom_ANN(nn.Module):
    
    def __init__(self, input_size, hidden_sizes, output_size):
        super(Custom_ANN, self).__init__()
        self.input_layer = nn.Linear(input_size, hidden_sizes[0])
        self.hidden_layers = nn.ModuleList([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]) for i in range(len(hidden_sizes)-1)])
        self.output_layer = nn.Linear(hidden_sizes[-1], output_size)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.input_layer(x))
        for layer in self.hidden_layers:
            x = self.relu(layer(x))
        x = self.output_layer(x)
        return x



In [7]:
input_size = bert_model.config.hidden_size  # Adjusted input size
hidden_sizes = [256, 128]
output_size = 2
ann_layers = Custom_ANN(input_size, hidden_sizes, output_size).to(device)

In [8]:
class CustomModel(nn.Module):
    def __init__(self, bert_model, ann_layers):
        super(CustomModel, self).__init__()
        self.bert_model = bert_model
        self.ann_layers = ann_layers
        
    def forward(self, input_ids, attention_mask):
        bert_output = self.bert_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = bert_output.pooler_output
        output = self.ann_layers(pooled_output)
        return output

In [9]:
model = CustomModel(bert_model, ann_layers).to(device)


In [10]:
dataset_texts = [
    
     "She sits by the window, watching the raindrops cascade down the glass in a mesmerizing dance. In the quiet of the evening, his words echo in her mind, a gentle reminder of the feelings he confessed not long ago. Despite the warmth of the room, a chill settles in her heart as she grapples with the weight of his admission. She knows she should respond, but the words elude her, buried beneath layers of uncertainty and doubt. And so, she remains silent, hoping he'll understand the unspoken rejection veiled behind her hesitant smile.",
    
     "Underneath the starlit sky, surrounded by the soft rustle of leaves in the gentle breeze, she feels a flutter of anticipation in her chest. His words hang in the air, laden with sincerity and vulnerability, and she finds herself captivated by the earnestness in his gaze. She knows this moment will be etched into the fabric of her memories forever. With a soft sigh, she reaches out, intertwining her fingers with his, a silent affirmation of the love that blossoms between them. In that fleeting touch, she finds the courage to let go of her fears and doubts, embracing the warmth of his affection with open arms. And as she leans in to meet his gaze, she whispers the words she's longed to say, 'Yes, I feel it too.'",
    
     "The cafe is alive with the hum of conversation and the aroma of freshly brewed coffee. Across the table, he sits, eyes filled with hope and longing, waiting for her response. She fidgets with her cup, avoiding his gaze as she searches for the right words. It's not that she doesn't care for him, she does, deeply. But love is a complicated thing, tangled in the threads of past experiences and future uncertainties. And so, with a heavy heart, she musters the courage to gently decline his offer, knowing that in doing so, she preserves their friendship and spares them both from potential heartache.",
    
     "In the golden glow of the setting sun, they walk hand in hand along the sandy shore, the sound of waves crashing against the rocks a soothing backdrop to their conversation. With each step, she feels the weight of his gaze upon her, filled with a tenderness that makes her heart swell with emotion. And as they pause to watch the sunset, painting the sky in hues of pink and orange, she knows that this moment is perfect, just as they are together. Turning to him, she smiles, her eyes shining with love and adoration, as she whispers the words he's been longing to hear, 'Yes, I'll be yours.'",
    
     "Amidst the hustle and bustle of the city, they find themselves alone in a quiet corner of the park. He takes her hand in his, his touch gentle yet filled with longing. She feels a knot form in her stomach as she realizes what he's about to say. She wants to reciprocate his feelings, she truly does, but something holds her back, a nagging doubt that refuses to be ignored. And so, with a heavy heart, she gently rejects his advances, knowing that in doing so, she's protecting both of their hearts from potential pain.",
    
     "Beneath the twinkling stars, they sit on the grassy hillside, the city lights shimmering in the distance. His words hang in the air, hesitant yet filled with sincerity, and she finds herself unable to look away from the vulnerability in his eyes. She knows that this moment will change everything, and she welcomes it with open arms. With a soft smile, she reaches out to touch his cheek, her heart overflowing with love as she whispers the words he's been longing to hear, 'Yes, I'll be yours forever.'",
    
     "In the quiet of the library, surrounded by shelves of books, he finally gathers the courage to confess his feelings. She listens intently, her heart heavy with the weight of his words. She wants to return his affections, she truly does, but there's a part of her that hesitates, a voice whispering doubts in the back of her mind. And so, with a sigh, she gently declines his offer, knowing that it's better to be honest now than to lead him on.",
    
     "As they dance beneath the soft glow of fairy lights, she feels a sense of contentment wash over her. His arms around her waist, pulling her close, make her feel safe and cherished. She knows that this is where she belongs, in his embrace, for now and always. And as the music fades into the night, she leans in to whisper the words he's been waiting to hear, 'Yes, I'll be your partner in this dance called life.'",
    
     "In the quiet of the night, he pours his heart out to her, laying bare his deepest emotions. She listens, her heart aching with the weight of his words. She wants to reciprocate his feelings, she truly does, but there's a voice inside her that holds her back, a fear of the unknown. And so, with a heavy heart, she gently turns him down, knowing that it's better to be honest now than to give him false hope.",
    
     "Beneath the canopy of stars, they sit on the hood of his car, sharing stories and dreams beneath the night sky. His hand finds hers, fingers intertwining in a silent promise of forever. She feels a warmth spread through her chest, a sense of belonging she's never known before. And as they gaze at the constellations above, she knows that this is where she's meant to be, with him by her side. With a smile, she leans in to press a kiss to his cheek, silently affirming the love that blooms between them.",
    
     "In the quiet of the park, surrounded by the gentle rustle of leaves, he finally gathers the courage to tell her how he feels. She listens, her heart heavy with the weight of his words. She wants to reciprocate his feelings, she truly does, but there's a part of her that holds back, a fear of what the future might hold. And so, with a sigh, she gently declines his offer, knowing that it's better to be honest now than to lead him on.",
    
     "She gazes out of the window, lost in thought as the rain taps against the glass in a rhythmic melody. His words linger in the air, heavy with expectation and vulnerability. She wants to reciprocate his feelings, to embrace the love he offers, but there's a hesitation that lingers in the depths of her heart. And so, with a soft smile and a gentle touch, she deflects his advances, hoping he'll understand the unspoken rejection hidden beneath her words.",
    
     "Beneath the starry sky, they find themselves enveloped in a blanket of silence, broken only by the soft whispers of the wind. His hand finds hers, fingers intertwining in a silent promise of forever. She feels a warmth spread through her chest, a sense of peace and belonging washing over her. And as they gaze at the constellations above, she knows that this is where she's meant to be, with him by her side. With a smile, she leans in to press a kiss to his lips, sealing their love beneath the watchful eye of the moon.",
    
     "In the quiet of the park, she listens to his confession with a heavy heart. His words hang in the air, filled with hope and longing, but she can't find it in herself to return his affections. There's a part of her that wants to, that yearns to make him happy, but she knows deep down that it wouldn't be fair to either of them. And so, with a gentle touch and a sad smile, she lets him down softly, hoping he'll understand the depth of her feelings.",
    
     "Amidst the hustle and bustle of the city, they find themselves alone on a rooftop, the city lights twinkling below. His eyes meet hers, filled with an intensity that takes her breath away. She knows in that moment that he's the one she wants to spend her life with. And as he reaches for her hand, she doesn't hesitate to say yes, her heart overflowing with love and excitement for the future they'll build together.",
    
     "As they stroll through the park, he gathers his courage and finally lays his feelings bare. She listens intently, her heart heavy with the weight of his words. She wants to reciprocate his affection, to make him happy, but there's a hesitation that holds her back. And so, with a soft smile and a gentle shake of her head, she gently declines his offer, knowing that it's for the best.",
    
     "In the quiet of the night, they sit by the fireplace, the crackling flames casting a warm glow over the room. His eyes search hers, filled with a love that makes her heart flutter. She knows in that moment that she's found her soulmate, the one she wants to share her life with. And as he reaches for her hand, she doesn't hesitate to say yes, her heart overflowing with joy and gratitude for the love they share.",
    
     "She listens to his declaration of love with a heavy heart, knowing that she can't return his feelings in the same way. There's a sadness in her eyes as she gently lets him down, her words carefully chosen to spare his feelings. She wishes things could be different, that she could love him the way he deserves, but she knows that it's not meant to be.",
    
     "Underneath the twinkling stars, they find themselves lost in each other's eyes, the world melting away around them. His words are like poetry, filling her with a sense of wonder and awe. She knows in that moment that she's found her other half, the one she's been searching for. And as he drops to one knee and presents her with a ring, she doesn't hesitate to say yes, tears of joy streaming down her face.",
    
     "In the dimly lit cafe, he pours his heart out to her, baring his soul in a way he's never done before. She listens intently, her heart aching with the weight of his words. She wishes she could love him the way he wants her to, but there's a part of her that knows it's not meant to be. And so, with a heavy heart and tears in her eyes, she gently turns him down, knowing that it's for the best.",
    
     "As they stand on the beach, waves crashing against the shore, he takes her hand in his and looks into her eyes with a love that takes her breath away. She feels a warmth spread through her chest, a sense of peace and contentment washing over her. And as he drops to one knee and asks her to be his forever, she doesn't hesitate to say yes, her heart overflowing with love and happiness."


]

dataset_labels = [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]


In [11]:
train_texts = dataset_texts
train_labels = torch.tensor(dataset_labels)
   
train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors='pt')
train_dataset = torch.utils.data.TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)



In [12]:
optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

In [18]:
import time

# Set the model to training mode
model.train()

# Iterate over epochs
for epoch in range(50):
    total_loss = 0
    true_positives, false_positives, false_negatives = 0, 0, 0

    # Initialize progress bar
    print(f'Epoch {epoch + 1}/{50} [', end='', flush=True)

    # Iterate over the batches in the training data loader
    for batch_idx, (input_ids, attention_mask, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        # Compute true positives, false positives, and false negatives
        predictions = outputs.argmax(dim=1)
        true_positives += ((predictions == 1) & (labels == 1)).sum().item()
        false_positives += ((predictions == 1) & (labels == 0)).sum().item()
        false_negatives += ((predictions == 0) & (labels == 1)).sum().item()

        # Update progress bar
        progress = int((batch_idx + 1) / len(train_loader) * 20)  # 20 is the width of the progress bar
        print('=' * progress, end='', flush=True)
        print('\r', end='')  # Move the cursor back to the beginning of the line
        time.sleep(0.1)  # Wait for 0.1 seconds between each update

    # Close progress bar
    print(']', end='', flush=True)
    print()  # Move to the next line for the next epoch

    # Calculate precision, recall
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Precision: {precision}, Recall: {recall}')


]poch 1/50 [====================
Epoch 1, Loss: 0.045107465237379074, Precision: 1.0, Recall: 1.0
]poch 2/50 [====================
Epoch 2, Loss: 0.04435582831501961, Precision: 1.0, Recall: 1.0
]poch 3/50 [====================
Epoch 3, Loss: 0.043670903891325, Precision: 1.0, Recall: 1.0
]poch 4/50 [====================
Epoch 4, Loss: 0.042815856635570526, Precision: 1.0, Recall: 1.0
]poch 5/50 [====================
Epoch 5, Loss: 0.04215378314256668, Precision: 1.0, Recall: 1.0
]poch 6/50 [====================
Epoch 6, Loss: 0.041503045707941055, Precision: 1.0, Recall: 1.0
]poch 7/50 [====================
Epoch 7, Loss: 0.04079173132777214, Precision: 1.0, Recall: 1.0
]poch 8/50 [====================
Epoch 8, Loss: 0.040221042931079865, Precision: 1.0, Recall: 1.0
]poch 9/50 [====================
Epoch 9, Loss: 0.039485715329647064, Precision: 1.0, Recall: 1.0
]poch 10/50 [====================
Epoch 10, Loss: 0.03887737914919853, Precision: 1.0, Recall: 1.0
]poch 11/50 [============

In [44]:
test_dataset = [
    # Tanishka and Gourav
    "Tanishka sits across from Gourav in the cozy corner of the cafe, his words hanging in the air like delicate tendrils of smoke. She wants to reciprocate his feelings, but there's a barrier between them, a wall built of unspoken fears and uncertainties. And so, with a gentle smile and a shake of her head, she softly declines his proposal, hoping he'll understand the depth of her hesitation.",
    "Gourav gazes into Tanishka's eyes, his heart pounding with anticipation as he waits for her response. When she finally nods, a radiant smile lighting up her face, he feels a surge of joy wash over him. He takes her hand in his, feeling grateful for this moment of happiness that they'll share together.",
    
    # Shukla and Raj
    "Shukla listens to Raj's heartfelt confession with a heavy heart, knowing that she can't return his feelings in the same way. There's a sadness in her eyes as she gently lets him down, her words carefully chosen to spare his feelings. She wishes things could be different, that she could love him the way he deserves, but she knows that it's not meant to be.",
    "Raj's heart races as he looks into Shukla's eyes, his hopes hanging on her every word. When she finally smiles and nods, he feels like he's on top of the world. He reaches for her hand, feeling grateful for this chance to build something beautiful together.",
    
    # Shreya and Anirban
    "Shreya feels a pang of guilt as Anirban pours his heart out to her, his words filled with vulnerability and longing. She knows she should reciprocate his feelings, but there's a part of her that holds back, a voice whispering doubts in the back of her mind. And so, with a heavy heart, she gently declines his offer, knowing that it's better to be honest now than to give him false hope.",
    "Anirban's heart skips a beat as Shreya's eyes meet his, filled with a warmth and affection that takes his breath away. When she finally nods, a smile spreading across her face, he feels like the luckiest person in the world. He takes her hand in his, feeling grateful for this chance to embark on a journey of love together.",
    
    # Ankuri and Rajdeep
    "Ankuri listens to Rajdeep's declaration of love with a heavy heart, knowing that she can't return his feelings in the same way. There's a sadness in her eyes as she gently lets him down, her words carefully chosen to spare his feelings. She wishes things could be different, that she could love him the way he deserves, but she knows that it's not meant to be.",
    "Rajdeep's heart swells with joy as Ankuri smiles and nods, her eyes shining with warmth and affection. He takes her hand in his, feeling grateful for this chance to build a future together filled with love and happiness.",
    
    # Purki and Souvik
    "Purki feels a knot form in her stomach as Souvik confesses his feelings, his words hanging in the air like a heavy cloud. She wants to reciprocate his affection, to make him happy, but there's a hesitation that holds her back. And so, with a sad smile and a gentle shake of her head, she gently turns him down, knowing that it's for the best.",
    "Souvik's heart races as Purki's eyes meet his, filled with a warmth and tenderness that takes his breath away. When she finally nods, a smile lighting up her face, he feels like he's on top of the world. He takes her hand in his, feeling grateful for this chance to build a future together filled with love and happiness.",
    
    # Urbi and Chayan
    "Urbi listens to Chayan's confession with a heavy heart, knowing that she can't return his feelings in the same way. There's a sadness in her eyes as she gently lets him down, her words carefully chosen to spare his feelings. She wishes things could be different, that she could love him the way he deserves, but she knows that it's not meant to be.",
    "Chayan's heart swells with joy as Urbi smiles and nods, her eyes shining with warmth and affection. He takes her hand in his, feeling grateful for this chance to build a future together filled with love and happiness."

    ,
    "In the soft glow of the evening sun, her gaze wandered, drifting over the words he had spoken with such vulnerability. There was a weight in her heart as she considered the depth of his feelings, like petals heavy with dew in the early morning light. With a gentle sigh, she began to speak, her voice a melody of hesitation and uncertainty, each word chosen with care. 'I appreciate your honesty,' she started, her eyes cast downward, 'and I value our friendship more than words can express.' A faint smile graced her lips, but it was tinged with sadness, as if she carried the weight of unspoken truths. 'But perhaps,' she continued, her voice barely above a whisper, 'we could explore this connection further, see where it leads us.' Though her words seemed to linger in the air like fragile whispers, there was a flicker of hope in her eyes, a glimmer of possibility hidden within the depths of her soul."
]

test_label = [0,1,0,1,0,1,0,1,0,1,0,1,1]



In [45]:
test_texts = test_dataset
test_labels = torch.tensor(test_label)
print(test_labels)

tensor([0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1])


In [46]:
test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors='pt')


In [47]:
test_dataset = torch.utils.data.TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [48]:
# Set the model to evaluation mode
model.eval()

all_preds = []
true_labels = []

# Iterate over the test data
with torch.no_grad():
    for input_ids, attention_mask, labels in test_loader:
        # Move data to GPU if available
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        
        # Perform forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        
        # Compute predictions
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.tolist())
        true_labels.extend(labels.tolist())

# Calculate accuracy
accuracy = accuracy_score(true_labels, all_preds)
print(f'Accuracy: {accuracy}')

# Print true and predicted labels
print("True Labels:", true_labels)
print("Predicted Labels:", all_preds)

Accuracy: 1.0
True Labels: [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1]
Predicted Labels: [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1]
